In [1]:
import json
import os, stat
import pandas as pd
import time
from datetime import datetime, timedelta

### Import tweetpy and set up API keys

In [2]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener


consumer_key = 'uTJUL5CmrGeh1MzwoTsSMvMRO'
consumer_secret = 'nDy5n8OXXxZI1IyiXEJNdNAOioWxGbHLRZHdGu7mZngxubkvdk'
access_token = '1569580854-6nITvc11aIHB5Vp6YSy1KJLQjsQ8NkldRigY80l'
access_secret = 'y6RojodWSKUduvSHmq6tl7z4n491kJqau9AcqH84F6uZv'


auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

### Check if the API is working

In [5]:
timeline = api.user_timeline(screen_name='MikePence')

In [6]:
len(timeline)

20

In [8]:
timeline[0]._json

{'created_at': 'Tue Apr 06 02:51:15 +0000 2021',
 'id': 1379265416728616962,
 'id_str': '1379265416728616962',
 'text': '@acnewsitics @mjhillenbrand coke would...fizzle?',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'acnewsitics',
    'name': 'Alex Cole',
    'id': 1080188052365029376,
    'id_str': '1080188052365029376',
    'indices': [0, 12]},
   {'screen_name': 'mjhillenbrand',
    'name': 'mjhillenbrand',
    'id': 16941994,
    'id_str': '16941994',
    'indices': [13, 27]}],
  'urls': []},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': 1379253716251865088,
 'in_reply_to_status_id_str': '1379253716251865088',
 'in_reply_to_user_id': 1080188052365029376,
 'in_reply_to_user_id_str': '1080188052365029376',
 'in_reply_to_screen_name': 'acnewsitics',
 'user': {'id': 4567901,
  'id_str': '4567901',
  'name': 'The Original',
  'screen_name': 'mikepence',
  'locati

### Load data of users you wish to pull tweets for. 
The field 'screen_name' contains the @-screen_name of the user. Preferably, you should also have the Twitter user_id

In [11]:
df_poli = pd.read_json('../data/usa_July_21.json')
screen_names = df_poli.screen_name.tolist()
df_poli.columns

Index(['id_str', 'id', 'screen_name', 'domain', 'state'], dtype='object')

In [12]:
len(screen_names)

8409

In [13]:
chunks = [screen_names[i:i+100] for i in range(0, len(screen_names), 100)]

In [20]:
_users = []
errors = {}
count=0
for c in chunks:
    try:
#         _user = api.lookup_users(user_ids=l_ids, include_entities=False)
        _user = api.lookup_users(screen_names=c)
        _users.extend(_user)
        print(count, len(_user), len(_users))
        count += 1
    except Exception as e:
        print(e, count)
        errors[s] = e
        count += 1
        continue

0 98 98
1 97 195
2 96 291
3 96 387
4 96 483
5 92 575
6 92 667
7 95 762
8 94 856
9 96 952
10 98 1050
11 95 1145
12 96 1241
13 97 1338
14 98 1436
15 95 1531
16 96 1627
17 98 1725
18 99 1824
19 97 1921
20 96 2017
21 100 2117
22 98 2215
23 97 2312
24 98 2410
25 96 2506
26 99 2605
27 98 2703
28 96 2799
29 97 2896
30 99 2995
31 94 3089
32 98 3187
33 96 3283
34 95 3378
35 99 3477
36 99 3576
37 97 3673
38 99 3772
39 97 3869
40 96 3965
41 95 4060
42 98 4158
43 99 4257
44 97 4354
45 94 4448
46 100 4548
47 99 4647
48 95 4742
49 97 4839
50 92 4931
51 99 5030
52 97 5127
53 98 5225
54 96 5321
55 99 5420
56 95 5515
57 98 5613
58 98 5711
59 97 5808
60 99 5907
61 97 6004
62 99 6103
63 94 6197
64 97 6294
65 99 6393
66 98 6491
67 97 6588
68 97 6685
69 94 6779
70 97 6876
71 94 6970
72 97 7067
73 98 7165
74 96 7261
75 95 7356
76 100 7456
77 98 7554
78 96 7650
79 95 7745
80 99 7844
81 94 7938
82 94 8032
83 98 8130
84 9 8139


In [21]:
errors

{}

In [22]:
len(errors)

0

In [23]:
df_users = pd.DataFrame([elem._json for elem in _users])
len(df_users)
# df_users.to_json('../data/twitter_us_candidates.json')

8139

In [24]:
df_users

,id,id_str,name,screen_name,location,description,url,entities,protected,followers_count,...,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type
0,239548513,239548513,Doug Jones,DougJones,"Birmingham, Alabama",Louise's husband. Father of three. Granddad of...,https://t.co/0jQtyaV9Ib,{'url': {'urls': [{'url': 'https://t.co/0jQtya...,False,266400,...,EFEFEF,333333,False,True,False,False,False,False,False,none
1,827258161841135623,827258161841135623,Secretary Ryan Zinke,SecretaryZinke,America,Former Secretary of the @Interior for @POTUS T...,https://t.co/1PxI1hPRg3,{'url': {'urls': [{'url': 'https://t.co/1PxI1h...,False,81671,...,000000,000000,False,False,False,False,False,False,False,none
2,24004390,24004390,Eric Greitens,EricGreitens,"Missouri, USA",Proud Dad • Navy SEAL • 56th Governor of Misso...,https://t.co/UuY4CCC8rz,{'url': {'urls': [{'url': 'https://t.co/UuY4CC...,False,94434,...,CCC6A7,000000,True,False,False,False,False,False,False,none
3,21111098,21111098,Richard Shelby,SenShelby,"Tuscaloosa, Alabama",The official Twitter page for U.S. Senator Ric...,https://t.co/1iULT8X6Vz,{'url': {'urls': [{'url': 'https://t.co/1iULT8...,False,93667,...,FFFFFF,333333,False,False,False,False,False,False,False,none
4,19668491,19668491,Governor Kay Ivey,GovernorKayIvey,"Montgomery, AL",54th Governor of Alabama,https://t.co/nkODVXq4YZ,{'url': {'urls': [{'url': 'https://t.co/nkODVX...,False,64798,...,E3E2DE,634047,True,True,False,False,False,False,False,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8134,28083041,28083041,Tom,Lubnau,"Wyoming, USA",lawyer/firefighter/state rep.,None,{'description': {'urls': []}},False,326,...,DDEEF6,333333,True,False,True,False,False,False,False,none
8135,97705249,97705249,Rita Meyer,ritaforgovernor,Wyoming,I’m running for Governor of this great state o...,http://t.co/psFmvQ9sdi,{'url': {'urls': [{'url': 'http://t.co/psFmvQ9...,False,158,...,DAECF4,663B12,True,False,False,False,False,False,False,none
8136,28717902,28717902,Tony Ross,Dagsgolf,"Cheyenne, Wyoming",Wyoming Senate Appropriations Chairman,None,{'description': {'urls': []}},False,146,...,95E8EC,3C3940,True,False,False,False,False,False,False,none
8137,152141149,152141149,Leslie Petersen,peeps4petersen,"Wilson, Wyoming",citizen of wyoming,None,{'description': {'urls': []}},False,65,...,EADEAA,333333,True,False,False,False,False,False,False,none


In [25]:
# df = pd.read_json('../data/twitter_us_candidates.json')
# df_users = df
# len(df)

### Set the in_directory and out_directory
Set the in_directory and out_directory variables. The in_directory refers to the directory that holds tweets that were pulled previously. The out_directory refers to the path where the new updated tweets collected in this run will be stored.

Note that the new tweets (collected in the current run) and the old tweets (stored in in_directory) will be combined (contatenated) and stored in out_directory. If the user does not exist in in_directory, the script will pull all tweets accessible using the API (the lates 3200 tweets of the user).

For the first run, the in_directory and out_directory can be the same (as there are no tweets in the in_directory).

In [26]:
in_directory = '/path/to/in_dir'
out_directory = '/path/to/out_dir'

In [27]:
def get_latest_tweet_id(screen_name):
    filename = screen_name + '.json'
    directory = in_directory
    df = pd.read_json(directory + filename)
    df = df.sort_values(['created_at'])
#     print df.created_at.tolist()[-1]
    return df.id_str.tolist()[-1], df

def get_all_recent_tweets(id_, from_id):
    count = 1
    all_tweets = []
    tweets_json = []
    tweets = api.user_timeline(user_id = id_, count = 200, since_id = from_id, tweet_mode='extended')
    if len(tweets) == 0:
        return []
    tweets_json = [elem._json for elem in tweets]
    all_tweets.extend(tweets_json)
    screen_name = tweets_json[0]['user']['screen_name']
    print('\t', count, len(all_tweets), screen_name)
    while True:
        until_id = all_tweets[-1]['id_str']
        tweets = api.user_timeline(user_id = id_, count = 200, max_id = until_id, since_id = from_id, tweet_mode='extended')
        count += 1
        if len(tweets) == 1:
            break
        tweets_json = [elem._json for elem in tweets]
        all_tweets.extend(tweets_json)
        
        print('\t', count, len(all_tweets), screen_name, until_id, tweets_json[0]['created_at'], tweets_json[-1]['created_at'])
    return all_tweets

def get_new_accounts(id_):
    count = 1
    all_tweets = []
    tweets_json = []
    tweets = api.user_timeline(user_id = id_, count = 200, tweet_mode='extended')
    if len(tweets) == 0:
        return []
    tweets_json = [elem._json for elem in tweets]
    all_tweets.extend(tweets_json)
    screen_name = tweets_json[0]['user']['screen_name']
    print('\t', count, len(all_tweets), screen_name)
    while True:
        until_id = all_tweets[-1]['id_str']
        tweets = api.user_timeline(user_id = id_, count = 200, max_id = until_id, tweet_mode='extended')
        count += 1
        if len(tweets) == 1:
            break
        tweets_json = [elem._json for elem in tweets]
        all_tweets.extend(tweets_json)
        
        print('\t', count, len(all_tweets), screen_name, until_id, tweets_json[0]['created_at'], tweets_json[-1]['created_at'])
    return all_tweets

def get_new_accounts_w_screen_name(screen_name):
    count = 1
    all_tweets = []
    tweets_json = []
    tweets = api.user_timeline(screen_name = screen_name, count = 200, tweet_mode='extended')
    if len(tweets) == 0:
        return []
    tweets_json = [elem._json for elem in tweets]
    all_tweets.extend(tweets_json)
    print('\t', count, len(all_tweets), screen_name, 'pulling with screen name')
    while True:
        until_id = all_tweets[-1]['id_str']
        tweets = api.user_timeline(screen_name = screen_name, count = 200, max_id = until_id, tweet_mode='extended')
        count += 1
        if len(tweets) == 1:
            break
        tweets_json = [elem._json for elem in tweets]
        all_tweets.extend(tweets_json)
        
        print('\t', count, len(all_tweets), screen_name, until_id, tweets_json[0]['created_at'], tweets_json[-1]['created_at'])
    return all_tweets

def get_screen_name(id_):
    return df[df.id_str == id_]['screen_name'].tolist()[0]

In [28]:
# current_accounts = [elem.split('.json')[0] for elem in select_files]
current_accounts = []
df = df_users

In [26]:
# Comment line below if you want to skip over accounts already pulled, 
# say if the script failed and needs to restart from the point of failure
# current_accounts = [elem.split('.json')[0] for elem in os.listdir(out_directory)]
start = time.time()
error_accounts = {}
count = 0
for id_ in df_users.id_str.tolist():
    s = get_screen_name(id_)
    print(count, s)
    count += 1
    if s in current_accounts:
        continue
    current_accounts.append(s)
#     count += 1
    try:
        from_id, df_old = get_latest_tweet_id(s)
    except Exception as e:
        try:
            tweets = get_new_accounts(id_)
        except Exception as e:
            try:
                tweets = get_new_accounts_w_screen_name(s)
            except Exception as e:
                error_accounts[s] = e
                print(e)
                continue
        if len(tweets) > 0:
            df_json = pd.DataFrame(tweets)
#             df_json.to_json(out_directory + s + '.json')
        continue
    try:
        tweets = get_all_recent_tweets(id_, from_id)
    except Exception as e:
        error_accounts[s] = e
        print(e)
        continue
    if len(tweets) > 0:
        df_new = pd.DataFrame(tweets)
        df_new.to_json('../tweets/temp.json')
        df_new = pd.read_json('../tweets/temp.json')
        df_json = pd.concat([df_old, df_new], sort=False).reset_index(drop=True)
        df_json.to_json(out_directory + s + '.json')
    else:
        df_json = df_old
        df_json.to_json(out_directory + s + '.json')
end = time.time()
print(end - start)

0 GenoveseSal
1 Team_Nunes
2 DougLaMalfa
Not authorized.
3 chaseforflorida
Not authorized.
4 yuforcongress
5 Barge4Congress
Not authorized.
6 Matt4GA
Not authorized.
7 ForPaulette
8 Maraforindiana
Not authorized.
9 Vanessa23607546
	 1 49 Vanessa23607546
10 VoteDelgado
Not authorized.
11 natalialinou
Not authorized.
12 JonHoadley
	 1 200 JonHoadley
	 2 400 JonHoadley 1343988056714571776 Tue Dec 29 18:31:37 +0000 2020 Fri Oct 23 01:50:10 +0000 2020
	 3 600 JonHoadley 1319456048579792899 Fri Oct 23 01:50:10 +0000 2020 Mon Oct 12 14:29:44 +0000 2020
	 4 678 JonHoadley 1315660930026549251 Mon Oct 12 14:29:44 +0000 2020 Wed Oct 07 13:19:42 +0000 2020
13 collincpeterson
Not authorized.
14 voterivera
Not authorized.
15 JonathanHerzog5
	 1 200 JonathanHerzog5
	 2 394 JonathanHerzog5 1345161343104667654 Sat Jan 02 00:13:50 +0000 2021 Mon Jun 29 01:08:15 +0000 2020
	 3 592 JonathanHerzog5 1277408505973129216 Mon Jun 29 01:08:15 +0000 2020 Wed Jun 17 19:44:15 +0000 2020
	 4 790 JonathanHerzog5 127

	 4 702 lisamurkowski 1321830480438841349 Thu Oct 29 15:05:19 +0000 2020 Wed Oct 07 20:16:40 +0000 2020
24 SenMurphyOffice
25 senrobportman
	 1 200 senrobportman
	 2 400 senrobportman 1371605177678069761 Mon Mar 15 23:32:12 +0000 2021 Tue Feb 09 17:42:55 +0000 2021
	 3 600 senrobportman 1359196091300925447 Tue Feb 09 17:42:55 +0000 2021 Mon Dec 28 03:03:40 +0000 2020
	 4 800 senrobportman 1343392140806791169 Mon Dec 28 03:03:40 +0000 2020 Fri Nov 27 23:39:44 +0000 2020
	 5 1000 senrobportman 1332469184920514562 Fri Nov 27 23:39:44 +0000 2020 Sun Oct 25 16:30:02 +0000 2020
	 6 1106 senrobportman 1320402246874574850 Sun Oct 25 16:30:02 +0000 2020 Thu Oct 08 15:00:45 +0000 2020
26 Kilili_Sablan
	 1 200 Kilili_Sablan
	 2 400 Kilili_Sablan 1265401551318913026 Tue May 26 21:56:54 +0000 2020 Sun Feb 16 21:52:45 +0000 2020
	 3 600 Kilili_Sablan 1229161722017124353 Sun Feb 16 21:52:45 +0000 2020 Tue Jul 23 13:54:28 +0000 2019
	 4 800 Kilili_Sablan 1153664678779904000 Tue Jul 23 13:54:28 +0000 2

	 2 400 RepMeijer 1365399392719687681 Fri Feb 26 20:32:37 +0000 2021 Thu Jan 07 22:15:28 +0000 2021
	 3 434 RepMeijer 1347305882355462147 Thu Jan 07 22:15:28 +0000 2021 Sun Jan 03 15:09:31 +0000 2021
61 RepLisaMcClain
	 1 200 RepLisaMcClain
	 2 385 RepLisaMcClain 1364349322272387079 Tue Feb 23 23:00:01 +0000 2021 Sat Jan 02 16:33:29 +0000 2021
62 RepFischbach
	 1 200 RepFischbach
	 2 229 RepFischbach 1352652905346707458 Fri Jan 22 16:22:38 +0000 2021 Sun Jan 03 17:33:45 +0000 2021
63 RepCori
	 1 200 RepCori
	 2 203 RepCori 1346480825680986112 Tue Jan 05 15:36:59 +0000 2021 Sun Jan 03 23:43:42 +0000 2021
64 RepRosendale
	 1 200 RepRosendale
	 2 244 RepRosendale 1353841071789371396 Mon Jan 25 23:03:59 +0000 2021 Mon Jan 04 00:38:20 +0000 2021
65 RepDeborahRoss
	 1 200 RepDeborahRoss
	 2 400 RepDeborahRoss 1367301417338347524 Thu Mar 04 02:30:35 +0000 2021 Thu Jan 14 17:46:36 +0000 2021
	 3 411 RepDeborahRoss 1349774934474694656 Thu Jan 14 17:46:36 +0000 2021 Mon Jan 11 20:03:34 +0000 202

### Sample some tweets and check they are being pulled and stored pulled.

In [24]:
tweets = get_new_accounts_w_screen_name('JoeBiden')

	 1 200 JoeBiden pulling with screen name
	 2 400 JoeBiden 1339297242633490433 Wed Dec 16 19:52:00 +0000 2020 Mon Nov 09 15:25:10 +0000 2020
	 3 600 JoeBiden 1325821740258365441 Mon Nov 09 15:25:10 +0000 2020 Sun Nov 01 13:42:00 +0000 2020
	 4 800 JoeBiden 1322896675040456704 Sun Nov 01 13:42:00 +0000 2020 Fri Oct 23 14:17:00 +0000 2020
	 5 1000 JoeBiden 1319643992523296769 Fri Oct 23 14:17:00 +0000 2020 Mon Oct 12 23:10:00 +0000 2020
	 6 1200 JoeBiden 1315791859919122433 Mon Oct 12 23:10:00 +0000 2020 Wed Sep 30 23:19:49 +0000 2020
	 7 1400 JoeBiden 1311445675197050881 Wed Sep 30 23:19:49 +0000 2020 Mon Sep 14 16:04:00 +0000 2020
	 8 1600 JoeBiden 1305537793414889472 Mon Sep 14 16:04:00 +0000 2020 Wed Aug 26 02:24:46 +0000 2020
	 9 1800 JoeBiden 1298446258944839680 Wed Aug 26 02:24:46 +0000 2020 Mon Aug 10 17:10:00 +0000 2020
	 10 2000 JoeBiden 1292870827168342017 Mon Aug 10 17:10:00 +0000 2020 Fri Jul 17 01:20:00 +0000 2020
	 11 2200 JoeBiden 1283934443057577984 Fri Jul 17 01:20:00 +

In [20]:
len(error_accounts)

22

In [ ]:
low_tweet_accounts = []
count = 0
start = datetime(2019,1,1)
end = datetime(2019,5,31)
for f in os.listdir(out_directory):
    df = pd.read_json(out_directory + f)
    df = df[(df.created_at >= start)&(df.created_at <= end)].reset_index(drop=True)
    if len(df) < 50:
        count += 1
        low_tweet_accounts.append(f)
    print(count, f)

In [ ]:
df

In [ ]:
df_errors = pd.DataFrame({'screen_name':list(error_accounts.keys()), 'error': list(error_accounts.values())})
# df_errors.to_json('../tweets/error_accounts/errors_28_May.json')
df_errors